## Descrizione

Il sistema BERT implementato in questo esperimento rappresenta l’approccio classico al problema della classificazione delle fake news.  
Si basa interamente sull’apprendimento automatico supervisionato: il modello viene addestrato su un dataset etichettato con notizie “vere” o “false”, e impara a riconoscere pattern testuali per ripetere questa classificazione.

## Limiti dell’approccio BERT

Questo metodo presenta diversi limiti:  
- Non riconosce le campagne di disinformazione, analizzando ogni notizia in modo isolato senza collegarla ad altre.  
- Non utilizza conoscenza esperta né dati strutturati.  
Nel nostro framework, invece, queste informazioni sono fornite da esperti, raccolte e organizzate per supportare chi fa analisi di sicurezza e difesa.

## Il mio approccio innovativo

Il framework proposto nella mia tesi adotta un approccio innovativo che integra modelli linguistici, conoscenza esperta e strumenti di cyber threat intelligence (CTI).  

In questo modo, non solo si riesce a rilevare le fake news, ma si ricostruisce anche chi le ha diffuse, con quale scopo e in quale contesto narrativo, rendendo l’analisi molto più utile e operativa in scenari reali.


## PARTE 1:
Il codice installa tutte le librerie necessarie (come transformers, torch, sklearn) e verifica che l’ambiente sia pronto. Inoltre, rileva se è disponibile una GPU per accelerare l’addestramento.

In [1]:
import subprocess
import sys
import warnings
import pandas as pd
import numpy as np
import urllib
import json
import re
import pickle
import torch
import time
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

warnings.filterwarnings('ignore')

# PARTE 1: INSTALLAZIONE DIPENDENZE
def install_requirements():
    """Installa le librerie necessarie per BERT"""
    requirements = [
        'transformers',
        'torch',
        'datasets',
        'accelerate',
        'scikit-learn',
        'pandas',
        'numpy'
    ]

    print("Installazione dipendenze per BERT Fake News Detection...")
    print("=" * 60)

    for package in requirements:
        try:
            __import__(package.replace('-', '_'))
            print(f"[OK] {package} già installato")
        except ImportError:
            print(f"Installando {package}...")
            try:
                subprocess.check_call([
                    sys.executable, "-m", "pip", "install", package, "--quiet"
                ])
                print(f"[OK] {package} installato")
            except subprocess.CalledProcessError as e:
                print(f"[ERRORE] Errore nell'installazione di {package}: {e}")

    print("\n" + "=" * 60)
    print("Installazione completata!")
    return True

def test_imports():
    """Test import librerie"""
    print("\nTest import librerie...")
    try:
        import transformers
        import torch
        import pandas as pd
        import numpy as np
        from sklearn.metrics import accuracy_score
        print("[OK] Tutte le librerie importate correttamente!")
        print(f"Transformers version: {transformers.__version__}")
        print(f"PyTorch version: {torch.__version__}")

        if torch.cuda.is_available():
            print(f"GPU disponibile: {torch.cuda.get_device_name()}")
        else:
            print("Utilizzo CPU")

        return True
    except ImportError as e:
        print(f"[ERRORE] Errore nell'import: {e}")
        return False

##PARTE 2: CARICAMENTO E PREPROCESSING DATI
Scarica i dataset di notizie vere e false da internet
Pulisce i testi (rimuove caratteri strani, mette tutto minuscolo)
Divide i dati in training/validation/test
Mette le etichette: 0=Vera, 1=Falsa

In [2]:
# PARTE 2: CARICAMENTO E PREPROCESSING DATI
def load_convert_data(url):
    """Scarica e converte i dati JSON"""
    try:
        print(f"Scaricando dati da: {url.split('/')[-1]}")
        with urllib.request.urlopen(url) as response:
            df = json.loads(response.read().decode())
            df = pd.DataFrame.from_dict(df)
        print(f"Caricati {len(df)} campioni")
        return df
    except Exception as e:
        print(f"[ERRORE] Errore nel caricamento da {url}: {e}")
        return None

def clean_txt(text):
    """Pulisce il testo rimuovendo caratteri speciali"""
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub("'", "", text)
    text = re.sub("(\\W)+", " ", text)
    text = text.lower().strip()
    return text

def load_and_prepare_datasets():
    """Carica e prepara tutti i dataset"""
    print("CARICAMENTO DATASET FAKE NEWS")
    print("=" * 50)

    urls = {
        'real_train': "https://storage.googleapis.com/public-resources/dataset/real_train.json",
        'real_test': "https://storage.googleapis.com/public-resources/dataset/real_test.json",
        'fake_train': "https://storage.googleapis.com/public-resources/dataset/fake_train.json",
        'fake_test': "https://storage.googleapis.com/public-resources/dataset/fake_test.json"
    }

    datasets = {}
    for name, url in urls.items():
        df = load_convert_data(url)
        if df is not None:
            datasets[name] = df
        else:
            print(f"[ERRORE] Impossibile caricare {name}")
            return None

    print("\nTutti i dataset caricati con successo!")

    # Aggiungi etichette
    datasets['real_train']['label'] = 0  # Real = 0
    datasets['real_test']['label'] = 0
    datasets['fake_train']['label'] = 1  # Fake = 1
    datasets['fake_test']['label'] = 1

    # Combina training e test
    train_df = pd.concat([datasets['real_train'], datasets['fake_train']], ignore_index=True)
    test_df = pd.concat([datasets['real_test'], datasets['fake_test']], ignore_index=True)

    print(f"Training set: {len(train_df)} campioni")
    print(f"Test set: {len(test_df)} campioni")

    # Pulizia testi
    print("Pulizia testi...")
    train_df['text'] = train_df['text'].apply(clean_txt)
    test_df['text'] = test_df['text'].apply(clean_txt)

    print("Preprocessing completato!")
    return train_df, test_df

def prepare_train_val_split(train_df, test_size=0.2, random_state=42):
    """Prepara split training/validation"""
    print(f"Creazione split training/validation ({test_size*100:.0f}% per validation)")

    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_df['text'].tolist(),
        train_df['label'].tolist(),
        test_size=test_size,
        random_state=random_state,
        stratify=train_df['label']
    )

    print(f"Training: {len(train_texts)} campioni")
    print(f"Validation: {len(val_texts)} campioni")

    return train_texts, val_texts, train_labels, val_labels

##PARTE 3: MODELLO BERT

In [3]:
class FakeNewsDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def initialize_bert_model(model_name='bert-base-uncased'):
    """Inizializza il modello BERT"""
    print(f"Caricamento modello {model_name}...")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=2,
            problem_type="single_label_classification"
        )

        print("Modello inizializzato correttamente!")

        total_params = sum(p.numel() for p in model.parameters())
        print(f"Parametri totali: {total_params:,}")

        if torch.cuda.is_available():
            model = model.cuda()
            print(f"Modello spostato su GPU: {torch.cuda.get_device_name()}")
        else:
            print("Modello su CPU")

        return model, tokenizer

    except Exception as e:
        print(f"[ERRORE] Errore nell'inizializzazione del modello: {e}")
        return None, None

def prepare_datasets(train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, tokenizer, max_length=512):
    """Prepara i dataset per il training"""
    print("Preparazione dataset per BERT...")

    train_dataset = FakeNewsDataset(train_texts, train_labels, tokenizer, max_length)
    val_dataset = FakeNewsDataset(val_texts, val_labels, tokenizer, max_length)
    test_dataset = FakeNewsDataset(test_texts, test_labels, tokenizer, max_length)

    print("Dataset preparati con successo!")
    return train_dataset, val_dataset, test_dataset


##PARTE 4: TRAINING E VALUTAZIONE

In [ ]:

def compute_metrics(eval_pred):
    """Calcola metriche per la valutazione"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(labels, predictions)
    return {'accuracy': accuracy}

def setup_trainer(model, tokenizer, train_dataset, val_dataset, output_dir="./bert_results"):
    """Configura il trainer per il training"""
    print("Configurazione trainer...")

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        learning_rate=2e-5,
        warmup_steps=500,
        weight_decay=0.01,
        logging_steps=100,
        eval_steps=500,
        save_strategy="steps",
        save_steps=1000,
        eval_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        remove_unused_columns=False,
        dataloader_pin_memory=False,
        fp16=torch.cuda.is_available(),
        report_to=None,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    print("Trainer configurato con successo!")
    return trainer

def train_model(trainer, output_dir="./bert_fake_news_model"):
    """Esegue il training del modello"""
    print("INIZIO TRAINING MODELLO BERT")
    print("=" * 60)

    start_time = time.time()

    try:
        print("Avvio training...")
        training_result = trainer.train()
        print("Training completato con successo!")

        training_time = time.time() - start_time
        print(f"Training completato in: {training_time/60:.1f} minuti")

        print(f"Salvataggio modello in {output_dir}...")
        trainer.save_model(output_dir)
        trainer.tokenizer.save_pretrained(output_dir)
        print("Modello salvato con successo!")

        return trainer, training_result, training_time

    except Exception as e:
        print(f"[ERRORE] Errore durante il training: {e}")
        return None, None, 0

def evaluate_model(trainer, test_dataset):
    """Valuta il modello sul test set"""
    print("VALUTAZIONE MODELLO SUL TEST SET")
    print("=" * 50)

    try:
        print("Esecuzione predizioni...")
        predictions = trainer.predict(test_dataset)

        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = predictions.label_ids

        accuracy = accuracy_score(y_true, y_pred)
        print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

        print("\nCLASSIFICATION REPORT:")
        print("-" * 40)
        report = classification_report(
            y_true, y_pred,
            target_names=['Real', 'Fake'],
            digits=4
        )
        print(report)

        print("\nCONFUSION MATRIX:")
        cm = confusion_matrix(y_true, y_pred)
        print(f"True Real, Pred Real: {cm[0,0]}")
        print(f"True Real, Pred Fake: {cm[0,1]}")
        print(f"True Fake, Pred Real: {cm[1,0]}")
        print(f"True Fake, Pred Fake: {cm[1,1]}")

        return {
            'accuracy': accuracy,
            'predictions': y_pred,
            'true_labels': y_true,
            'confusion_matrix': cm,
            'classification_report': report
        }

    except Exception as e:
        print(f"[ERRORE] Errore nella valutazione: {e}")
        return None

def test_predictions(model, tokenizer, test_texts, test_labels, num_examples=5):
    """Testa il modello con alcuni esempi"""
    print(f"TEST PREDIZIONI SU {num_examples} ESEMPI")
    print("=" * 60)

    for i in range(min(num_examples, len(test_texts))):
        text = test_texts[i]
        true_label = test_labels[i]

        print(f"\nEsempio {i+1}:")
        print(f"Testo: {text[:150]}...")
        print(f"Etichetta vera: {'FAKE' if true_label == 1 else 'REAL'}")

        inputs = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=-1).item()
            confidence = predictions[0][predicted_class].item()

        print(f"Predizione: {'FAKE' if predicted_class == 1 else 'REAL'}")
        print(f"Confidenza: {confidence:.4f}")
        print(f"Corretto: {'SI' if predicted_class == true_label else 'NO'}")

def analyze_custom_news(model, tokenizer):
    """Analizza notizie inserite dall'utente"""
    print("ANALISI NOTIZIE PERSONALIZZATE")
    print("=" * 60)
    print("Inserisci una notizia per verificare se è vera o falsa.")
    print("Digita 'quit' per terminare.\n")

    while True:
        try:
            print("-" * 40)
            news_text = input("Inserisci la notizia da analizzare:\n> ").strip()

            if news_text.lower() in ['quit', 'exit', 'q']:
                print("Analisi terminata!")
                break

            if not news_text or len(news_text) < 10:
                print("Inserisci un testo valido (almeno 10 caratteri).")
                continue

            print("Analisi in corso...")

            inputs = tokenizer(
                news_text,
                truncation=True,
                padding='max_length',
                max_length=512,
                return_tensors='pt'
            )

            if torch.cuda.is_available():
                inputs = {k: v.cuda() for k, v in inputs.items()}

            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
                predicted_class = torch.argmax(predictions, dim=-1).item()
                confidence = predictions[0][predicted_class].item()
                real_confidence = predictions[0][0].item()
                fake_confidence = predictions[0][1].item()

            print("\nRISULTATI ANALISI:")
            print("=" * 30)
            print(f"Testo: {news_text[:100]}{'...' if len(news_text) > 100 else ''}")
            print(f"\nPREDIZIONE: {'FAKE NEWS' if predicted_class == 1 else 'NOTIZIA VERA'}")
            print(f"Confidenza: {confidence:.4f} ({confidence*100:.2f}%)")
            print(f"\nDETTAGLIO:")
            print(f"   Probabilità VERA: {real_confidence:.4f} ({real_confidence*100:.2f}%)")
            print(f"   Probabilità FAKE: {fake_confidence:.4f} ({fake_confidence*100:.2f}%)")

        except KeyboardInterrupt:
            print("\nAnalisi interrotta!")
            break
        except Exception as e:
            print(f"[ERRORE] Errore: {e}")

def quick_test_mode():
    """Modalità rapida per testare solo notizie personalizzate"""
    print("MODALITÀ TEST RAPIDO")
    print("=" * 40)

    try:
        model_paths = ["./bert_fake_news_model", "bert_fake_news_model"]
        model_path = None

        for path in model_paths:
            if os.path.exists(path):
                model_path = path
                break

        if model_path is None:
            print("[ERRORE] Modello pre-allenato non trovato!")
            print("Esegui prima il training completo")
            return

        print(f"Caricamento modello da: {model_path}")

        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSequenceClassification.from_pretrained(model_path)

        if torch.cuda.is_available():
            model = model.cuda()
            print("Modello su GPU")
        else:
            print("Modello su CPU")

        print("Modello caricato con successo!")
        analyze_custom_news(model, tokenizer)

    except Exception as e:
        print(f"[ERRORE] Errore nel caricamento: {e}")

def main():
    """Funzione principale completa"""
    print("BERT FAKE NEWS DETECTION SYSTEM")
    print("=" * 60)

    # Parte 1: Installazione
    print("\nPARTE 1: INSTALLAZIONE DIPENDENZE")
    install_requirements()
    if not test_imports():
        return

    # Parte 2: Caricamento dati
    print("\nPARTE 2: CARICAMENTO E PREPROCESSING DATI")
    train_df, test_df = load_and_prepare_datasets()
    if train_df is None or test_df is None:
        return

    train_texts, val_texts, train_labels, val_labels = prepare_train_val_split(train_df)
    test_texts = test_df['text'].tolist()
    test_labels = test_df['label'].tolist()

    # Parte 3: Inizializzazione modello
    print("\nPARTE 3: INIZIALIZZAZIONE MODELLO BERT")
    model, tokenizer = initialize_bert_model()
    if model is None or tokenizer is None:
        return

    train_dataset, val_dataset, test_dataset = prepare_datasets(
        train_texts, train_labels, val_texts, val_labels, test_texts, test_labels, tokenizer
    )

    # Parte 4: Training e valutazione
    print("\nPARTE 4: TRAINING E VALUTAZIONE")
    trainer = setup_trainer(model, tokenizer, train_dataset, val_dataset)

    trainer, training_result, training_time = train_model(trainer)
    if trainer is None:
        return

    evaluation_results = evaluate_model(trainer, test_dataset)
    if evaluation_results:
        print(f"\nAccuracy finale: {evaluation_results['accuracy']:.4f} ({evaluation_results['accuracy']*100:.2f}%)")

    print(f"Tempo totale training: {training_time/60:.1f} minuti")

    # Test esempi
    test_predictions(model, tokenizer, test_texts, test_labels)

    # Test interattivo
    print("\nFASE INTERATTIVA")
    print("=" * 40)
    analyze_custom_news(model, tokenizer)

if __name__ == "__main__":
    print("Scegli modalità:")
    print("1. Training completo + Test interattivo")
    print("2. Solo test interattivo (modello pre-allenato)")

    choice = input("\nInserisci il numero (1 o 2): ").strip()

    if choice == '2':
        quick_test_mode()
    else:
        main()


Scegli modalità:
1. Training completo + Test interattivo
2. Solo test interattivo (modello pre-allenato)

Inserisci il numero (1 o 2): 1
BERT FAKE NEWS DETECTION SYSTEM

PARTE 1: INSTALLAZIONE DIPENDENZE
Installazione dipendenze per BERT Fake News Detection...
[OK] transformers già installato
[OK] torch già installato
[OK] datasets già installato
[OK] accelerate già installato
Installando scikit-learn...
[OK] scikit-learn installato
[OK] pandas già installato
[OK] numpy già installato

Installazione completata!

Test import librerie...
[OK] Tutte le librerie importate correttamente!
Transformers version: 4.52.4
PyTorch version: 2.6.0+cu124
GPU disponibile: Tesla T4

PARTE 2: CARICAMENTO E PREPROCESSING DATI
CARICAMENTO DATASET FAKE NEWS
Scaricando dati da: real_train.json
Caricati 800 campioni
Scaricando dati da: real_test.json
Caricati 200 campioni
Scaricando dati da: fake_train.json
Caricati 800 campioni
Scaricando dati da: fake_test.json
Caricati 200 campioni

Tutti i dataset carica

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss


Training completato con successo!
Training completato in: 4.5 minuti
Salvataggio modello in ./bert_fake_news_model...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Modello salvato con successo!
VALUTAZIONE MODELLO SUL TEST SET
Esecuzione predizioni...


Accuracy: 0.9700 (97.00%)

CLASSIFICATION REPORT:
----------------------------------------
              precision    recall  f1-score   support

        Real     0.9563    0.9850    0.9704       200
        Fake     0.9845    0.9550    0.9695       200

    accuracy                         0.9700       400
   macro avg     0.9704    0.9700    0.9700       400
weighted avg     0.9704    0.9700    0.9700       400


CONFUSION MATRIX:
True Real, Pred Real: 197
True Real, Pred Fake: 3
True Fake, Pred Real: 9
True Fake, Pred Fake: 191

Accuracy finale: 0.9700 (97.00%)
Tempo totale training: 4.5 minuti
TEST PREDIZIONI SU 5 ESEMPI

Esempio 1:
Testo: march 6 that s when it all became very real i had just played my first basketball game in months the night before and conversations were swirling abou...
Etichetta vera: REAL
Predizione: REAL
Confidenza: 0.9990
Corretto: SI

Esempio 2:
Testo: as russias regional governments have been given the freedom to apply their own approaches to coronavirus 

#### Conclusioni

Il sistema BERT funziona bene con notizie simili a quelle su cui è stato addestrato.  
Ma quando incontra testi nuovi o diversi, spesso sbaglia perché si basa solo su semplici pattern di parole, senza capire davvero il significato.
Il mio framework, invece, usa conoscenza esperta e analisi semantica profonda, così può trovare non solo fake news, ma anche capire chi le ha create e perché.  

Nell'esempio ho testato:
1. Notizia proveniente da dataset di addestramento e riconosciuta come fake (Roger Stone suggested on Monday that Bill Gates might have ....)
1. Notizia proveniente da dataset di addestramento e riconosciuta come (Scots GPs told not to meet fever patients as fears of coronavirus outbreak grow ..)
1. Notizia proveniente fake proveniente dal nostro FakeCTI(Happy Valentine’s Day, everyone–especially Hillary Clinton...)-->Viene riconosciuta, invece, come true. ERRATO!
